# Introduction
Nous allons ici développer un algorithme de Machine Learning destiné à assigner automatiquement plusieurs tags pertinents à une question posée sur le célébre site Stack overflow.     
Ce programme s'adresse principalement aux nouveaux utilisateurs, afin de leur suggérer quelques tags relatifs à la question qu'ils souhaitent poser.

### Les données sources
Les données ont été cleanées. Dans ce nettoyage ont par exemple été appliquées les techniques de stop words, suppression de la ponctuation et des liens, tokenisation, lemmatisation ...

### Objectif de ce Notebook
Dans ce Notebook, nous allons traiter la partie modélisation des données textuelles avec des modèles supervisés et non supervisés.     

In [1]:
# Install package for PEP8 verification
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

Looking in indexes: https://test.pypi.org/simple/


In [2]:
!pip install gensim

In [3]:
# Import Python libraries
import os
import warnings
import time
import joblib
import numpy as np
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
from IPython.core.display import display, HTML

#RNN
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras import backend as K

# Library for PEP8 standard
from nbpep8.nbpep8 import pep8

C:\Users\jonat\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
C:\Users\jonat\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:627: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
C:\Users\jonat\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:637: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `

In [4]:
warnings.filterwarnings('ignore', category=DeprecationWarning)
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
set_config(display='diagram')

C:\Users\jonat\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
data = pd.read_csv("StackOverflow_questions_2009_2020_cleaned.csv", sep=";", index_col=0,
                   converters={"Title": literal_eval,
                               "Body": literal_eval,
                               "Tags": literal_eval}) 
data.head(3)

,Title,Body,Score,Tags
Id,,,,
120,"[aspnet, site, map]","[experience, sql, site, map, provider, default...",21,"[sql, asp.net]"
180,"[function, color, wheel]","[pseudo, time, solution, problem, way, color, ...",53,[algorithm]
260,"[scripting, functionality, net, application]","[game, database, end, trading, card, game, fun...",49,"[c#, .net]"


In [6]:
data.shape

(49603, 4)

Nous allons également créer une variable 'Full_doc' qui accueillera le document complet de chaque item (Title et Body) :

In [7]:
data["Full_doc"] = data["Title"] + data["Body"]
data["Full_doc"].head(3)

Id
120    [aspnet, site, map, experience, sql, site, map...
180    [function, color, wheel, pseudo, time, solutio...
260    [scripting, functionality, net, application, g...
Name: Full_doc, dtype: object

In [8]:
data.head(3)

,Title,Body,Score,Tags,Full_doc
Id,,,,,
120,"[aspnet, site, map]","[experience, sql, site, map, provider, default...",21,"[sql, asp.net]","[aspnet, site, map, experience, sql, site, map..."
180,"[function, color, wheel]","[pseudo, time, solution, problem, way, color, ...",53,[algorithm],"[function, color, wheel, pseudo, time, solutio..."
260,"[scripting, functionality, net, application]","[game, database, end, trading, card, game, fun...",49,"[c#, .net]","[scripting, functionality, net, application, g..."


In [9]:
def remove_tfidf (x):
    
    txt = " ".join(x)
    #txt = txt.replace('[', '')
    #txt = txt.replace(']', '')
    #txt = txt.replace(',', '')
    
    return txt

In [10]:
data['Full_doc'] = data['Full_doc'].apply(remove_tfidf)

In [11]:
data.head(3)

,Title,Body,Score,Tags,Full_doc
Id,,,,,
120,"[aspnet, site, map]","[experience, sql, site, map, provider, default...",21,"[sql, asp.net]",aspnet site map experience sql site map provid...
180,"[function, color, wheel]","[pseudo, time, solution, problem, way, color, ...",53,[algorithm],function color wheel pseudo time solution prob...
260,"[scripting, functionality, net, application]","[game, database, end, trading, card, game, fun...",49,"[c#, .net]",scripting functionality net application game d...


# Preprocessing : Bag of Words / Tf-Idf

Pour alimenter les modèles de machine learning, nous avons besoin de traiter des données numériques. Le modèle Bag of Words apprend un vocabulaire à partir de tous les documents, puis modélise chaque document en comptant le nombre de fois où chaque mot apparaît, convertissant donc les données textuelles en données numériques.

Nos données ayant déjà été cleanées et tokenisées, nous allons initialiser l'algorithme du CountVectorizer sur les variables Title et Body (X1 et X2) sans preprocessing. Enfin, nous allons utiliser le module TfidfVectorizer de la librairie Scikit-Learn pour combiner le CountVectorizer et TfidfTransformer. Cela aura pour effet de pondérer la fréquence d'apparition des mots par un indicateur de similarité (si ce mot est commun ou rare dans tous les documents). Dans cette partie, nous allons éliminer les mots qui apparaissent dans plus de 75% des documents (max_df = 0.75) ainsi que ceux qui apparaissent dans moins de 5% (min_df = 0.05).

La métrique tf-idf (Term-Frequency - Inverse Document Frequency) utilise comme indicateur de similarité l'Inverse Document Frequency,  qui est l'inverse de la proportion de document qui contient le terme, à l'échelle logarithmique.

Pour préparer nos targets (pour les modèles supervisés), nous allons utiliser MultiLabelBinarizer de Scikit-Learn puisque nos Tags sont multiples.

In [12]:
def tfidf(data):
    
    tfidf_vec = TfidfVectorizer()
    
    #tfidf_model = tfidf_vec.fit(data)
    
    #print(tfidf_model.dtype)
    
    #X_tfidf = tfidf_model.transform(data)
    
    X_tfidf = tfidf_vec.fit_transform(data)
    #X_feature = tfidf_vec.get_feature_names()
    return X_tfidf

def tfid2(data):
    
    vectorizer = CountVectorizer() # This class will transform the words in the text into a word frequency matrix, and the matrix element a[i][j] represents the word frequency of word j under class i text
    transformer = TfidfTransformer() # This class will count the tf-idf weights of each word
    tfidf_before = vectorizer.fit_transform(data)
    tfidf = transformer.fit_transform(tfidf_before)
    
    return tfidf

def tfidf3(X_train,X_test):
    
    tfidf_vec = TfidfVectorizer(analyzer="word",
                                max_df=0.75,
                                min_df=0.05,
                                tokenizer=None,
                                preprocessor=' '.join,
                                stop_words=None,
                                lowercase=False)
    
    X_train_tfidf = tfidf_vec.fit_transform(X_train)
    X_test_tfidf = tfidf_vec.transform(X_test)
    
    return X_train_tfidf,X_test_tfidf

In [13]:
# Define X and y
X = data["Full_doc"]
y = data["Tags"]

In [14]:
X_df = pd.DataFrame(y)
X_df.head(5)

,Tags
Id,
120,"[sql, asp.net]"
180,[algorithm]
260,"[c#, .net]"
330,[c++]
470,[.net]


In [15]:
y_df = pd.DataFrame(y)
y_df.head(5)

,Tags
Id,
120,"[sql, asp.net]"
180,[algorithm]
260,"[c#, .net]"
330,[c++]
470,[.net]


In [16]:
X.head(3)

Id
120    aspnet site map experience sql site map provid...
180    function color wheel pseudo time solution prob...
260    scripting functionality net application game d...
Name: Full_doc, dtype: object

In [17]:
X_tfidf = tfidf(X)

print("Shape of X for Full_doc: {}".format(X_tfidf.shape))

# Multilabel binarizer for targets
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(y)
y_binarized = multilabel_binarizer.transform(y)

print("Shape of y: {}".format(y_binarized.shape))

Shape of X for Full_doc: (49603, 47003)
Shape of y: (49603, 50)


On réduit maintenant les dimension à l'aide de SVD.
On souhaite conserver 95% de la variance.

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [19]:
svd = TruncatedSVD(n_components = 10000)

In [20]:
# Create train and test split (30%)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binarized, test_size=0.3, random_state=8)
print("X_train shape : {}".format(X_train.shape))
print("X_test shape : {}".format(X_test.shape))
print("y_train shape : {}".format(y_train.shape))
print("y_test shape : {}".format(y_test.shape))

X_train shape : (34722, 47003)
X_test shape : (14881, 47003)
y_train shape : (34722, 50)
y_test shape : (14881, 50)


In [ ]:
import scipy as sp

svd.fit(sp.sparse.vstack((X_train, X_test)))
X_train = svd.transform(X_train)
X_test = svd.transform(X_test)

# Modèles non supervisés

## Modèle LDA
LDA, ou Latent Dirichlet Allocation est un modèle probabiliste qui, pour obtenir des affectations de cluster, utilise deux valeurs de probabilité : P(word | topics) et P(topics | documents). Ces valeurs sont calculées sur la base d'une attribution aléatoire initiale, puis le calcul est répété pour chaque mot dans chaque document, pour décider de leur attribution de sujet. Dans cette méthode itérative, ces probabilités sont calculées plusieurs fois, jusqu'à la convergence de l'algorithme.

Nous allons entrainer 1 seul modèle basé sur la variable Full_doc en utilisant la librairie spécialisée Gensim. Pour cette partie, nous n'utiliserons pas le preprocessing TFIDF mais des fonctions propres aux méthodes Gensim.

Dans une première étape, le Bag of words est créé ainsi que la matrice de fréquence des termes dans les documents :

In [ ]:
# Define X and y
X = data["Full_doc"]
y = data["Tags"]

In [ ]:
# Create dictionnary (bag of words)
id2word = corpora.Dictionary(X)
id2word.filter_extremes(no_below=4, no_above=0.75, keep_n=None)
# Create Corpus 
texts = X  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

Gensim crée un identifiant unique pour chaque mot du document puis mappe word_id et word_frequency. Exemple : (6,3) ci-dessus indique que word_id 6 apparaît 3 fois dans le document et ainsi de suite.      
Les mots les plus fréquents ont ici aussi été filtrés grâce à la fonction filter_extremes réglée à 75% comme pour le Tfidf.

Pour voir quel mot correspond à un identifiant donné, il faut transmettre l'identifiant comme clé du dictionnaire. Exemple : id2word[4] :

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

Nous allons à présent entrainer le modèle LDA sur Full_doc puis afficher les métriques : 
- Perplexity :  Log likelihood - Densité de vraisemblance
- Coherence Score : Les mesures de cohérence de topics évaluent un seul topic en mesurant le degré de similitude sémantique entre les mots à score élevé dans ce dernier.

In [ ]:
# Build LDA model
full_lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=20,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
# Print Perplexity score
print('\nPerplexity: ', full_lda_model.log_perplexity(corpus))

#Print Coherence Score
coherence_model_lda = CoherenceModel(model=full_lda_model, 
                                     texts=texts, 
                                     dictionary=id2word, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### Visualisation des résultats de LDA Gensim sur Full_doc avec 20 topics

D'après les résultats de cette modélisation LDA, il semble très difficile de "nommer" les topics créés car les mots qui les composent sont très variés et sans fil conducteur clairement établi. On voit cependant par exemple que le topic représenté par "server" englobe également "database", "sql", "connection" ou encore "query" ce qui est cohérent.

### Amélioration du modèle LDA

Cependant, dans l'algoritme LDA, nous avons fixé arbitrairement à 20 le paramètre num_topics, qui représente le nombre de topics à créer. 
Afin de sélectionner le meilleur nombre de topics pour nos données, nous allons itérer sur un intervalle de nombre de topics et tester le score de cohérence pour chaque modèle :

In [ ]:
pyLDAvis.enable_notebook()
%matplotlib inline

display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))

gensimvis.prepare(full_lda_model, corpus, id2word)

In [ ]:
# Iter LDA for best number of topics
coherence_test = []
for k in np.arange(1,90,10):
    print("Fitting LDA for K = {}".format(k))
    start_time = time.time()
    lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=k,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
    coherence_model_lda = CoherenceModel(model=lda_model,
                                         texts=texts,
                                         dictionary=id2word,
                                         coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    end_time = time.time()
    coherence_test.append((k, coherence_lda,
                           (end_time - start_time)))

Affichons les scores des divers modèles :

In [ ]:
# Create dataframe of results
coherence_test = pd.DataFrame(coherence_test,
                              columns=["k","coherence","time"])

# Select best number of topics
best_nb_topics = coherence_test\
                    .loc[coherence_test.coherence.argmax(),"k"]

# Plot results
fig, ax1 = plt.subplots(figsize=(12,8))
x = coherence_test["k"]
y1 = coherence_test["coherence"]
y2 = coherence_test["time"]

ax1.plot(x, y1, label="Coherence score")
ax1.axvline(x=best_nb_topics, color='r', alpha=.7,
            linestyle='dashdot', label='Best param')
ax1.set_xlabel("Number of components")
ax1.set_ylabel("Coherence score")

ax2 = ax1.twinx()
ax2.plot(x, y2, label="Fit time",
         color='g', alpha=.5,
         linestyle='--')
ax2.set_ylabel("Fitting time (s)")

plt.title("Modèle LDA Optimal \n",
          color="#641E16", fontsize=18)
legend = fig.legend(loc=1, bbox_to_anchor=(.92, .9))

fig.tight_layout()
plt.show()

In [ ]:
best_nb_topics

Testons à présent le modèle avec le meilleur nombre théorique de topics pour l'afficher avec LDAvis :

In [ ]:
# Best LDA visualization
# Construire le modèle LDA
best_lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=best_nb_topics,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
gensimvis.prepare(best_lda_model, corpus, id2word)

Pour attribuer des Tags à chaque question sur ces modèles non-supervisés, nous allons créer une matrice Topic/Tags en réalisant une multiplication matricielle des matrices Document / Topic et Document / Tags.

In [ ]:
# Calculate Document/topic matrix with Gensim
doc_topic = pd.DataFrame(best_lda_model\
                             .get_document_topics(corpus,
                                                  minimum_probability=0))
for topic in doc_topic.columns:
    doc_topic[topic] = doc_topic[topic].apply(lambda x : x[1])

print('document/tag : ', y_binarized.shape)
print('document/topic : ', doc_topic.shape)

In [ ]:
# Print documents / topic matrix
doc_topic.head(3)

A présent, créons la matrice Topic / Tags grâce aux probabilités obtenues :

In [ ]:
# Matricial multiplication with Document / Topics transpose
topic_tag = np.matmul(doc_topic.T, y_binarized)
topic_tag.shape

In [ ]:
topic_tag

Nous obtenons donc une matrice dont les lignes représentent les Topics créés et les colonnes les Tags associés et leurs distribution. Nous allons donc créer nos prédictions en prenant les n premiers tags associés aux topics de chaque document :

In [ ]:
y_results = pd.DataFrame(y)
y_results["best_topic"] = doc_topic.idxmax(axis=1).values
y_results["nb_tags"] = y_results["Tags"].apply(lambda x : len(x))

df_y_bin = pd.DataFrame(y_binarized)
df_dict = dict(
    list(
        df_y_bin.groupby(df_y_bin.index)
    )
)

tags_num = []
for k, v in df_dict.items():
    check = v.columns[(v == 1).any()]
    tags_num.append(check.to_list())

y_results["y_true"] = tags_num
y_results.head(3)

In [ ]:
# Select predicted tags in Topics / Tags matrix
list_tag = []
for row in y_results.itertuples():
    nb_tags = row.nb_tags
    best_topic = row.best_topic
    row_tags = list(topic_tag.iloc[best_topic]\
                    .sort_values(ascending=False)[0:nb_tags].index)
    list_tag.append(row_tags)
    
y_results["y_pred"] = list_tag
y_results.head(3)

Nous allons tester plusieurs métriques sur ce modèle LDA :
- Accuracy score :
- F1 score :
- Jaccard similarity score : 
- Recall :
- Precision :

In [ ]:
def metrics_score(model, df, y_true, y_pred):
    """Compilation function of metrics specific to multi-label
    classification problems in a Pandas DataFrame.
    This dataFrame will have 1 row per metric
    and 1 column per model tested. 

    Parameters
    ----------------------------------------
    model : string
        Name of the tested model
    df : DataFrame 
        DataFrame to extend. 
        If None : Create DataFrame.
    y_true : array
        Array of true values to test
    y_pred : array
        Array of predicted values to test
    ----------------------------------------
    """
    if(df is not None):
        temp_df = df
    else:
        temp_df = pd.DataFrame(index=["Accuracy", "F1",
                                      "Jaccard", "Recall",
                                      "Precision"],
                               columns=[model])
        
    scores = []
    scores.append(metrics.accuracy_score(y_true, 
                                         y_pred))
    scores.append(metrics.f1_score(y_pred, 
                                   y_true, 
                                   average='weighted'))
    scores.append(metrics.jaccard_score(y_true, 
                                        y_pred, 
                                        average='weighted'))
    scores.append(metrics.recall_score(y_true, 
                                       y_pred, 
                                       average='weighted'))
    scores.append(metrics.precision_score(y_true, 
                                          y_pred, 
                                          average='weighted'))
    temp_df[model] = scores
    
    return temp_df

In [ ]:
# Create matrix for pred and true y LDA
lda_y_pred = np.zeros(y_binarized.shape)
n = 0
for row in y_results.y_pred.values:
    for i in range(len(row)):
        lda_y_pred[n,row[i]] = 1
    n+=1
    
lda_y_true = np.zeros(y_binarized.shape)
m = 0
for row in y_results.y_true.values:
    for i in range(len(row)):
        lda_y_true[m,row[i]] = 1
    m+=1

In [ ]:
df_metrics_compare = metrics_score("LDA", df=None,
                                   y_true=lda_y_true,
                                   y_pred=lda_y_pred)
df_metrics_compare

On remarque ici que la modélisation non supervisée avec LDA n'est pas adaptée. En effet, le meilleur nombre de topics se situerait à 21, mais l'algorithme ne parvient pas a établir de groupes bien distincts. Un certain nombre de topics sont très regroupés et donc représentés par les mêmes termes.

# Modèles supervisés

## Régression logistique avec multi-labels

Cette technique vise à construire un modèle permettant de prédire / expliquer les valeurs prises par une variable cible qualitative à partir d’un ensemble de variables explicatives quantitatives ou qualitatives encodées.

Pour cette partie sur les modélisations supervisées, nous allons utiliser la variable Full_doc qui regroupe le Title et le Body puis créer un Pipeline qui ne pourra pas inclure la transformation de notre variable cible (MultiLabelBinarizer ne fonctionne pas dans les Pipeline SKlearn).

In [ ]:
# Initialize Logistic Regression with OneVsRest
param_logit = {"estimator__C": [100, 10, 1.0, 0.1],
               "estimator__penalty": ["l1", "l2"],
               "estimator__dual": [False],
               "estimator__solver": ["liblinear"]}

multi_logit_cv = GridSearchCV(OneVsRestClassifier(LogisticRegression()),
                              param_grid=param_logit,
                              n_jobs=-1,
                              cv=5,
                              scoring="f1_weighted",
                              return_train_score = True,
                              refit=True)
multi_logit_cv.fit(X_train, y_train)

In [ ]:
logit_cv_results = pd.DataFrame.from_dict(multi_logit_cv.cv_results_)
print("-"*50)
print("Best params for Logistic Regression")
print("-" * 50)
logit_best_params = multi_logit_cv.best_params_
print(logit_best_params)

In [ ]:
logit_cv_results[logit_cv_results["params"]==logit_best_params]

Nous pouvons maintenant réaliser les prédictions avec le modèle de régression logistique sur le jeu de test pour pouvoir les comparer avec le jeu y_test.

In [ ]:
# Predict
y_test_predicted_labels_tfidf = multi_logit_cv.predict(X_test)

# Inverse transform
y_test_pred_inversed = multilabel_binarizer\
    .inverse_transform(y_test_predicted_labels_tfidf)
y_test_inversed = multilabel_binarizer\
    .inverse_transform(y_test)

print("-"*50)
print("Print 5 first predicted Tags vs true Tags")
print("-" * 50)
print("Predicted:", y_test_pred_inversed[0:5])
print("True:", y_test_inversed[0:5])

Puis nous calculons les diverses métriques sur le meilleur modèle de régression logistique :

In [ ]:
df_metrics_compare = metrics_score("Logit", 
                                   df=df_metrics_compare, 
                                   y_true = y_test,
                                   y_pred = y_test_predicted_labels_tfidf)
df_metrics_compare

## <span id="section_3_2">Modélisation avec RandomForest</span>

In [ ]:
# Initialize RandomForest with OneVsRest
param_rfc = {"estimator__max_depth": [5, 25, 50],
             "estimator__min_samples_leaf": [1, 5, 10],
             "estimator__class_weight": ["balanced"]}

multi_rfc_cv = GridSearchCV(OneVsRestClassifier(RandomForestClassifier()),
                            param_grid=param_rfc,
                            n_jobs=-1,
                            cv=2,
                            scoring="f1_weighted",
                            return_train_score = True,
                            refit=True,
                            verbose=3)
# Fit on Sample data
multi_rfc_cv.fit(X_train[0:7000], y_train[0:7000])

In [ ]:
rfc_cv_results = pd.DataFrame.from_dict(multi_rfc_cv.cv_results_)
print("-"*50)
print("Best params for RandomForestClassifier")
print("-" * 50)
rfc_best_params = multi_rfc_cv.best_params_
print(rfc_best_params)

In [ ]:
rfc_best_params_ok = {}
for k, v in rfc_best_params.items():
    rfc_best_params_ok[k.replace("estimator__","")] = v

In [ ]:
# Refit RandomForestClassifier best_params with full dataset
rfc_final_model = OneVsRestClassifier(RandomForestClassifier(**rfc_best_params_ok))
rfc_final_model.fit(X_train, y_train)

# Predict
y_test_predicted_labels_tfidf_rfc = rfc_final_model.predict(X_test)

# Inverse transform
y_test_pred_inversed_rfc = multilabel_binarizer\
    .inverse_transform(y_test_predicted_labels_tfidf_rfc)

print("-"*50)
print("Print 5 first predicted Tags vs true Tags")
print("-" * 50)
print("Predicted:", y_test_pred_inversed_rfc[0:5])
print("True:", y_test_inversed[0:5])

In [ ]:
df_metrics_compare = metrics_score("RandomForest", 
                                   df=df_metrics_compare,
                                   y_true = y_test,
                                   y_pred = y_test_predicted_labels_tfidf_rfc)
df_metrics_compare

Les métriques sur le modèle RandomForest sont moins bonnes mais semblent cependant plus cohérente avec les données, ce d'autant que les métriques Jaccard et F1 sont proches. D'autre part, nous pouvons vérifier le nombre de lignes dont les Tags ne sont pas prédit afin de voir si l'un des modèles est meilleur :

In [ ]:
Tags_per_row_lr = y_test_predicted_labels_tfidf.sum(axis=1)
null_rate_lr = round(((Tags_per_row_lr.size - np.count_nonzero(Tags_per_row_lr))
                      /Tags_per_row_lr.size)*100,2)
Tags_per_row_rfc = y_test_predicted_labels_tfidf_rfc.sum(axis=1)
null_rate_rfc = round(((Tags_per_row_rfc.size - np.count_nonzero(Tags_per_row_rfc))
                       /Tags_per_row_rfc.size)*100,2)
print("-"*50)
print("Percentage of non tagged question for each model")
print("-" * 50)
print("Logistic Regression: {}%".format(null_rate_lr))
print("Random Forest: {}%".format(null_rate_rfc))

Le Random Forest semble donc plus approprié à notre programme d'auto-tagging sur les données Stackoverflow. Nous allons à présent tester ce modèle RandomForest avec Classifier Chains pour remplacer la méthode One versus rest :

## Modèle RandomForest avec Classifier Chains

Avec la méthode ClassifierChains, chaque modèle fait une prédiction dans l'ordre spécifié en utilisant toutes les fonctionnalités disponibles fournies au modèle mais également les prédictions des modèles précédents.

In [ ]:
rfc_base_model = RandomForestClassifier(**rfc_best_params_ok)
chain = ClassifierChain(rfc_base_model, order='random')
chain.fit(X_train, y_train)

In [ ]:
# Predict
y_test_predicted_labels_tfidf_chain = chain.predict(X_test)

# Inverse transform
y_test_pred_inversed_chain = multilabel_binarizer\
    .inverse_transform(y_test_predicted_labels_tfidf_chain)

print("-"*50)
print("Print 5 first predicted Tags vs true Tags")
print("-" * 50)
print("Predicted:", y_test_pred_inversed_chain[0:5])
print("True:", y_test_inversed[0:5])

In [ ]:
Tags_per_row_chain = y_test_predicted_labels_tfidf_chain.sum(axis=1)
null_rate_chain = round(((Tags_per_row_chain.size - np.count_nonzero(Tags_per_row_chain))
                       /Tags_per_row_chain.size)*100,2)
print("-"*50)
print("Percentage of non tagged question for chain model")
print("-" * 50)
print("RandomForest with Classifier Chains: {}%".format(null_rate_chain))

In [ ]:
df_metrics_compare = metrics_score("RFC Chains", 
                                   df=df_metrics_compare,
                                   y_true = y_test,
                                   y_pred = y_test_predicted_labels_tfidf_chain)
df_metrics_compare

Le modèle RandomForest avec Classifier Chains offre des métriques similaires au modèle avec OneVsRest mais le taux de remplissage des valeurs prédites est encore meilleur.

Nous allons à présent tester un dernier modèle d'apprentissage profond avec Keras et un réseau de neurones simple.

## <span id="section_3_4">Réseau de neurones avec Keras</span>

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def jaccard_m(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

Nous allons définir une fonction pour constuire le réseau de neurones assez simple. RNN avec une couche cachée et complétement connectée. Nous utiliserons également un Dropout pour éviter le sur-apprentissage.

In [ ]:
def build_nn(input_dim, hidden_neurons, output_dim):
    """
    Construct a Keras model which will be used to 
    fit/predict in SKlearn pipeline.
    """
    # Create brain
    model = Sequential()
    model.add(layers.Dense(hidden_neurons,
                           input_dim=input_dim,
                           activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(hidden_neurons,
                           input_dim=input_dim,
                           activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(output_dim,
                           activation='softmax'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', recall_m, precision_m, f1_m, jaccard_m])
    model.summary()
    
    return model

In [ ]:
clear_session()

model_params = {
    'input_dim': X_train.shape[1],
    'hidden_neurons': 150,
    'output_dim': y_train.shape[1]}

keras_model = build_nn(**model_params)

In [ ]:
y_train.shape[1]

In [ ]:
history = keras_model.fit(X_train, y_train,
                          epochs=20,
                          batch_size=256,
                          verbose=0,
                          validation_data=(X_test, y_test),
                          shuffle=True)

In [ ]:
# evaluate the model
scores = keras_model.evaluate(X_test, y_test)
df_metrics_compare["Keras NN"] = [scores[i] for i in [1,4,5,2,3]]
df_metrics_compare

In [ ]:
train_accuracy = history.history.get('accuracy', [])
train_f1 = history.history.get('f1_m', [])
val_accuracy = history.history.get('val_accuracy', [])
val_f1 = history.history.get('val_f1_m', [])

fig, axes = plt.subplots(1, 2, figsize=(25, 8))
axes[0].plot(np.arange(0,20,1),
             train_accuracy,
             label="Train")
axes[0].plot(np.arange(0,20,1),
             val_accuracy,
             linestyle='--', color='g', alpha=.7,
             label="Validation")
axes[0].set_xticks(np.arange(0,20,5))
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Accuracy score")
axes[0].set_title('Model accuracy through epochs',
                  color='#f48023', fontweight='bold')
axes[0].legend(loc=4)

axes[1].plot(np.arange(0,20,1),
             train_f1, label="Train")
axes[1].plot(np.arange(0,20,1),
             val_f1,
             linestyle='--', color='g', alpha=.7,
             label="Validation")
axes[1].set_xticks(np.arange(0,20,5))
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("F1 score")
axes[1].set_title('Model F1 score through epochs',
                  color='#f48023', fontweight='bold')
axes[1].legend(loc=4)

plt.show()

In [ ]:
# Make prediction with Keras Model
y_test_predicted_labels_tfidf_keras = keras_model.predict(X_test)
y_test_predicted_labels_tfidf_keras = np.where(y_test_predicted_labels_tfidf_keras >= 0.5, 1, 0)

In [ ]:
# Inverse transform
y_test_pred_inversed_keras = multilabel_binarizer\
    .inverse_transform(y_test_predicted_labels_tfidf_keras)

print("-"*50)
print("Print 5 first predicted Tags vs true Tags")
print("-" * 50)
print("Predicted:", y_test_pred_inversed_keras[0:5])
print("True:", y_test_inversed[0:5])

Tags_per_row_keras = y_test_predicted_labels_tfidf_keras.sum(axis=1)
null_rate_keras = round(((Tags_per_row_keras.size - np.count_nonzero(Tags_per_row_keras))
                       /Tags_per_row_keras.size)*100,2)
print("\n")
print("-"*50)
print("Percentage of non tagged question for Keras model")
print("-" * 50)
print("Keras model: {}%".format(null_rate_keras))

# Sélection du modèle final

Pour sélectionner le modèle final, nous allons nous baser sur les scores obtenus aux métriques Jaccard et F1. Nous prendrons également en compte le taux de "non prédits" pour trouver le meilleur compromis entre tout ces indicateurs de performance.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = np.arange(len(df_metrics_compare.columns))
width = 0.35

fig = plt.figure(figsize=(18,10))
ax1 = fig.add_subplot(111)
f1_scores = ax1.bar(x - width/2, df_metrics_compare.iloc[1,:], width, label="F1 score")
jacc_scores = ax1.bar(x + width/2, df_metrics_compare.iloc[2,:], width, label="Jaccard score")

ax2 = ax1.twinx()
non_predict = ax2.plot(x, [0,30.08,4.45,1.45,21.88],
                       linestyle='--',
                       color="red", alpha=.7,
                       label='No predict (%)')
ax2.grid(None)

ax1.set_ylabel('Scores')
ax1.set_title('Comparaison des scores par modèles \n',
              color="#641E16", 
              fontdict={'fontsize': 30})
ax1.set_xticks(x)
ax1.set_xticklabels(df_metrics_compare.columns)

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc=0,
           fontsize=15)

fig.tight_layout()

plt.show()

## API

In [ ]:
# Export fitted model and Preprocessor
joblib.dump(multi_logit_cv,'logit_nlp_model.pkl')
joblib.dump(vectorizer,'tfidf_vectorizer.pkl')
joblib.dump(multilabel_binarizer,'multilabel_binarizer.pkl')

In [ ]:
from flask import Flask, request, redirect, url_for, flash, jsonify
import numpy as np
import pickle as p
import json


app = Flask(__name__)


@app.route('/api/', methods=['POST'])
def makecalc():
    data = request.get_json()
    prediction = np.array2string(model.predict(data))

    return jsonify(prediction)

if __name__ == '__main__':
    modelfile = 'logit_nlp_model.pkl'
    model = joblib.load(open(modelfile, 'rb'))
    app.run(debug=True, host='0.0.0.0')

In [ ]:
import requests
import json

url = 'http://0.0.0.0:5000/api/'

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=j_data, headers=headers)
print(r, r.text)

In [ ]:
import requests

headers = {'Content-type': 'application/json'}
response = requests.get('http://0.0.0.0:5000/api/',headers=headers)
print("-"*50)
print("API test response :")
print("-"*50)
print(response.json())